# Scraping Job info from LinkedIn

<a href="https://medium.com/nerd-for-tech/linked-in-web-scraper-using-selenium-15189959b3ba">Tutorial here.</a>


In [6]:
import sys
sys.path.append('..')

import time
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from src.bot.Scraper import Scraper
from src.database.csv import save_to_csv

## Initialisation

In [7]:
RAW_FOLDER = '../data/raw/'

In [8]:
# Create a scraper object
scraper = Scraper(delay=1)

# Login to LinkedIn
scraper.login()

## Search jobs

### Debugging

In [9]:
# Search jobs by title and location
job_title = 'data scientist'
job_location = 'canada'
scraper.search_jobs(job_title, job_location)

# Get the  pagination buttons
time.sleep(2)
pagination_buttons = scraper.get_pagination_buttons()

jobs = scraper.get_current_page_jobs()
pagination = scraper.driver.find_element(By.CLASS_NAME, "jobs-search-pagination__pages")
pagination_buttons = pagination.find_elements(By.XPATH, './/button')
pagination_buttons[0].click()

Found 5 search bars
Found 4 pagination buttons


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=137.0.7151.122)
Stacktrace:
	GetHandleVerifier [0x0x7ff7fd7fcda5+78885]
	GetHandleVerifier [0x0x7ff7fd7fce00+78976]
	(No symbol) [0x0x7ff7fd5b99fc]
	(No symbol) [0x0x7ff7fd611c64]
	(No symbol) [0x0x7ff7fd603654]
	(No symbol) [0x0x7ff7fd638b8a]
	(No symbol) [0x0x7ff7fd602f06]
	(No symbol) [0x0x7ff7fd638da0]
	(No symbol) [0x0x7ff7fd66122f]
	(No symbol) [0x0x7ff7fd638963]
	(No symbol) [0x0x7ff7fd6016b1]
	(No symbol) [0x0x7ff7fd602443]
	GetHandleVerifier [0x0x7ff7fdad4eed+3061101]
	GetHandleVerifier [0x0x7ff7fdacf33d+3037629]
	GetHandleVerifier [0x0x7ff7fdaee592+3165202]
	GetHandleVerifier [0x0x7ff7fd81730e+186766]
	GetHandleVerifier [0x0x7ff7fd81eb3f+217535]
	GetHandleVerifier [0x0x7ff7fd8059b4+114740]
	GetHandleVerifier [0x0x7ff7fd805b69+115177]
	GetHandleVerifier [0x0x7ff7fd7ec368+10728]
	BaseThreadInitThunk [0x0x7ffee836259d+29]
	RtlUserThreadStart [0x0x7ffee99aaf78+40]


### Collect Job Data

Search for jobs and loop through all pages to get the data. 

In [2]:
def collect_job_data(job_title, job_location, delay=1):
    
    # Search jobs by title and location
    scraper.search_jobs(job_title, job_location)
    time.sleep(delay)

    # Get the  pagination buttons
    pagination_buttons = scraper.get_pagination_buttons()
    print(f"{len(pagination_buttons)} pages found")

    # Loop through each page
    # [:-1] is to ignore the last page as we click on the i+1 button
    all_jobs = []
    for i, button in enumerate(pagination_buttons[:-1]):

        try:
            print("button", button)
            # get the jobs of the current page
            current_page_jobs = scraper.get_current_page_jobs()
            all_jobs.append(current_page_jobs)

            # navigate to the next page
            # button.click()
            # Get the buttons again to avoid a StaleElementReferenceException
            pagination_buttons = scraper.get_pagination_buttons()
            pagination_buttons[i+1].click()

        except StaleElementReferenceException as e:
            print('The was an error...')
            print(e)
        except Exception as e:
            print('The was an error...')
            print(e)

        return all_jobs

In [3]:
# def flatten_jobs_array(nested_job_data):
#     flattened_jobs = []

#     for job_group in nested_job_data:
#         if job_group is None:
#             flattened_jobs.append([None, None, None, None])
#             continue

#         for job_entry in job_group:
#             if job_entry is None:
#                 flattened_jobs.append([None, None, None, None])
#                 continue

#             job_details = []

#             for field in job_entry:
#                 try:
#                     if isinstance(field, WebElement):
#                         job_details.append(field)  # Extract and clean text
#                     else:
#                         job_details.append(field)
#                 except Exception:
#                     job_details.append(field)

#             flattened_jobs.append(job_details)

#     return flattened_jobs


from src.utils.utils import flatten_jobs_array

### Run the search

In [4]:
job_titles = ["data scientist", "llm engineer", "ai engineer", "machine learning engineer", "mlops engineer", "ai developer", "generative ai engineer"]
job_location = "canada"
all_jobs = []

In [5]:
for title in job_titles:
    try:
        print(f"Collecting data for {title}...")
        all_jobs = collect_job_data(title, job_location, delay=2)
        flattened_jobs = flatten_jobs_array(all_jobs)
        columns = ["Job Title", "Company", "City", "Work Mode", "Description"]

        save_to_csv(
            data=flattened_jobs, 
            folder=RAW_FOLDER, 
            filename=title,
            colnames=columns
        )
    except Exception as e:
        print(f"An error occurred while collecting data for {title}:")
        print(e)


An error occurred while collecting data for data scientist:
name 'scraper' is not defined
An error occurred while collecting data for llm engineer:
name 'scraper' is not defined
An error occurred while collecting data for ai engineer:
name 'scraper' is not defined
An error occurred while collecting data for machine learning engineer:
name 'scraper' is not defined
An error occurred while collecting data for mlops engineer:
name 'scraper' is not defined
An error occurred while collecting data for ai developer:
name 'scraper' is not defined
An error occurred while collecting data for generative ai engineer:
name 'scraper' is not defined


In [ ]:
scraper.close_driver()